In [2]:
!pwd

/content


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os
os.chdir("drive/My Drive/textcnn_updated/textcnn")

In [41]:
!ls

clean_data_abstract_ln_200.pkl	      NCBI_corpus
clean_data_disease_first_last.pkl     NCBI_corpus.zip
clean_data_superclass_3_4.pkl	      NCBItrainset.csv
clean_data_superclass_4_5.pkl	      network.py
clean_data_superclass_first_2.pkl     predictions-Copy1.ipynb
clean_data_superclass_first_last.pkl  predictions.ipynb
clean_data_superclass_last_2.pkl      __pycache__
clean_data.txt			      random.csv
components.py			      random.gsheet
dropout_layers.py		      results_bilstm_superclass.csv
first_review.png		      results_cnn_disease_names.csv
glove.42B.300d.txt		      results_cnn_superclass.csv
glove.42B.300d.txt.1		      text.txt
glove.42B.300d.zip		      Untitled.ipynb
glove.42B.300d.zip.1		      utils
linked_diseases_abstract.csv


In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14925621918643286441, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9822929732254715746
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17853612670086921580
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11054556775
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18100597747050925864
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Input
from keras.layers.embeddings import Embedding

from sklearn.preprocessing import MultiLabelBinarizer
import csv,pandas,re
from keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
from keras.layers import Embedding
import numpy as np
from keras.models import Model

import time,os
import components as com

# Torch imports
import torch
import torch.nn as nn
from torch import optim
import network as net
import traceback
from utils import tensor_utils as tu

from tqdm import tqdm

import pickle
import json
import requests
from pprint import pprint

device = torch.device("cuda")
cpu = torch.device("cpu")

Using TensorFlow backend.


In [0]:
debug = True
MAX_SEQUENCE_LENGTH = 200



class Timer:
    """ Simple block which can be called as a context, to know the time of a block. """
    def __enter__(self):
        self.start = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end = time.perf_counter()
        self.interval = self.end - self.start

In [0]:
from __future__ import unicode_literals, print_function
import spacy

nlp = spacy.load('en')
# Use this command to download the english language model if error comes. ! python -m spacy download en

In [0]:
def get_sentences_from_abstract(abstract):
    doc = nlp(abstract)
    sentences = [sent.string.strip() for sent in doc.sents]
    
    return sentences

def keep_sentences(sentences, index, start=2, end=4, sentence_type='last'):
#     new_abstract = []
    if sentence_type == 'first':
        new_abstract = sentences[:index]
    elif sentence_type == 'last':
        new_abstract = sentences[-index:]
    elif sentence_type == 'middle':
        new_abstract = sentences[start:end]
    elif sentence_type == 'first_last':
        new_abstract = sentences[:start] + sentences[-end:]
    
    return new_abstract

def join_lists(new_abstract):
    new_abstract = ''.join(map(str, new_abstract))
#     new_abstract.join('.')
    return new_abstract

In [0]:
#data helpers - copied from https://github.com/bhaveshoswal/CNN-text-classification-keras
def clean_str(string):
    """
    Tokenization/string cleaning for datasets.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ", string)
    string = re.sub(r"\)", " ", string)
    string = re.sub(r"\?", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()
  
  
  
def _get_class_for_doi_(doi:str) -> list:
    url = "http://id.nlm.nih.gov/mesh/sparql"
    query = """
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
                PREFIX owl: <http://www.w3.org/2002/07/owl#>
                PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
                PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
                PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
                PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
                PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
                PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
                PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
                SELECT DISTINCT ?p ?label ?uri 
                WHERE { mesh:%s meshv:broaderDescriptor* ?uri .
                    ?uri rdfs:label ?p.
                    FILTER NOT EXISTS{
                    ?uri meshv:broaderDescriptor ?x
                    }
                }
                """ % doi
    querystring = {"query":query, 'format':'json'}
    payload = ""
    headers = {
        'cache-control': "no-cache",
        'Postman-Token': "c9ace615-1b99-4caf-bcad-97ce1060973e"
        }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    return [x['uri']['value'] for x in response.json()['results']['bindings']]


def get_classes_for_dois(dois:list) -> list:
    cls = []
    for doi in dois:
        cls += _get_class_for_doi_(doi)
    
    return list(set(cls))

In [0]:
def get_terms_of_classes(list_terms) -> list:
    url = "http://id.nlm.nih.gov/mesh/sparql"
    query = """
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
            PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
            PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
            PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
            PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
            PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
            PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
            SELECT DISTINCT ?class
            WHERE { mesh:%s rdfs:label ?class . }
            """%list_terms
    querystring = {"query":query, 'format':'json'}
    payload = ""
    headers = {
        'cache-control': "no-cache",
        'Postman-Token': "c9ace615-1b99-4caf-bcad-97ce1060973e"
        }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    #jsonRes = response.json()
    
        
    return [x['class']['value'] for x in response.json()['results']['bindings']]

In [0]:
def get_list_for_terms(dois:list) -> list:
    cls = []
    for doi in dois:
      #print("doi: ", doi)
      cls += get_terms_of_classes(doi)
    
    return list(set(cls))

In [0]:
df = pandas.read_csv('linked_diseases_abstract.csv')
data = [x for x in df.to_records(index=False)] 
def read_till_next(data,index):
    temp = []
    temp_loc = 0
    for d in data[index+1:]:
        if 'a' in d[1] or 't' in d[1]:
            return temp,temp_loc
        else:
            temp_loc = temp_loc + 1
            temp.append([d[2],d[3],d[4]])
    
    return temp,temp_loc

loc = 0
final_data = []
for index,d in enumerate(data):
    if '|t|' in d[1]:
        continue
    elif loc != 0:
        loc = loc - 1
        continue
    else:
        if '|a|' in d[1]:
            temp_holder = {
                'abstract' : d[1],
                'keys' : []
            }
            keys,loc = read_till_next(data,index)
            temp_holder['keys'] = keys
            final_data.append(temp_holder)
clean_data = []
for f in final_data:
    sent = get_sentences_from_abstract(abstract=f['abstract'].split('|')[2])    # get sentences
    sent_keep = keep_sentences(sentences=sent, index=2, start=1, end=1,sentence_type='last' )  # sentences to keep
    abst = join_lists(new_abstract=sent_keep)   # as we get the list of sentences to keep, joining them as one string
#     abstract = clean_str(f['abstract'].split('|')[2])
    abstract = clean_str(abst)
    disease = []
    for k in f['keys']:
        disease.append(k[-1])
    clean_data.append((abstract,list(set(disease))))

In [14]:
clean_data[0]

('at least 75 of these tumours possess mutations affecting the amino terminal segment , normally involved in phosphorylation dependent , ubiquitin mediated degradation of the protein this percentage of ctnnb1 mutations is greater than in all other human tumours examined thus far , and directly implicates beta catenin lef misregulation as the major cause of hair matrix cell tumorigenesis in humans',
 ['D009369', 'D003110', 'D003110|D009369', 'D012878', 'D011125', 'D018296'])

In [15]:
#Now chanign the hiearchy of the diseases dataset where we only consider Diseases ontology 
clean_data_upper_class = []
us = []
for c in tqdm(clean_data):
    try:
        labels = []
        for l in c[1]:
            #print(l)
            if 'D' == l[0]:
                labels = labels + l.replace('+','|').split("|")
                #labels = labels + l.split('+')
         #change the class hiearchy 
        #new_labels = list(set(get_classes_for_dois(labels)))
        clean_data_upper_class.append((c[0], list(set(labels))))
    except:
        us.append(c)

clean_data = clean_data_upper_class

100%|██████████| 793/793 [00:00<00:00, 145136.06it/s]


In [16]:
clean_data[0]

('at least 75 of these tumours possess mutations affecting the amino terminal segment , normally involved in phosphorylation dependent , ubiquitin mediated degradation of the protein this percentage of ctnnb1 mutations is greater than in all other human tumours examined thus far , and directly implicates beta catenin lef misregulation as the major cause of hair matrix cell tumorigenesis in humans',
 ['D009369', 'D003110', 'D012878', 'D011125', 'D018296'])

In [0]:
import pickle

with open('clean_data_disease_last_2.pkl', 'wb') as file:
    pickle.dump(clean_data, file) 

In [18]:
ls

clean_data_abstract_ln_200.pkl        linked_diseases_abstract.csv
clean_data_disease_first_2.pkl        NCBI_corpus/
clean_data_disease_first_last.pkl     NCBI_corpus.zip
clean_data_disease_last_2.pkl         NCBItrainset.csv
clean_data_superclass_3_4.pkl         network.py
clean_data_superclass_4_5.pkl         predictions-Copy1.ipynb
clean_data_superclass_first_2.pkl     predictions.ipynb
clean_data_superclass_first_last.pkl  __pycache__/
clean_data_superclass_last_2.pkl      random.csv
clean_data.txt                        random.gsheet
components.py                         results_bilstm_superclass.csv
dropout_layers.py                     results_cnn_disease_names.csv
first_review.png                      results_cnn_superclass.csv
glove.42B.300d.txt                    text.txt
glove.42B.300d.txt.1                  Untitled.ipynb
glove.42B.300d.zip                    utils/
glove.42B.300d.zip.1


In [0]:
#creating id map of class that is each label is given a unique id
all_class = list(set([g for f in clean_data for g in f[1]]))
all_class_to_id = {}
for index,value in enumerate(all_class):
    all_class_to_id[value] = index

texts = [f[0] for f in clean_data]

#Tokenize,idfy,pad
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [20]:
#constructing the label matrix 
label_matrix = np.zeros((len(all_class),len(all_class)))
for index,value in enumerate(all_class):
    label_matrix[index][index] = 1
    
#Testing 
label_matrix[all_class_to_id[all_class[1]]]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [21]:
len(label_matrix[all_class_to_id[all_class[1]]])

578

In [0]:
def create_multi_hot_label(label_matrix,labels):
    '''
    
        label_matrix --> matrix having one hot vectors
        labels --> [idfy]--> [2,5,6]
        return [0,0,1,0,0,1....]
    '''
    vector = np.zeros((label_matrix.shape[0]))
    for i in labels:
        vector = vector + label_matrix[all_class_to_id[all_class[i]]]
    
    return vector

In [0]:
label_vector = []
for f in clean_data:
    label_id = [all_class_to_id[l] for l in f[1]]
    multi_hot_vector = create_multi_hot_label(label_matrix,label_id)
    label_vector.append(multi_hot_vector)

In [0]:
X = data
Y = label_vector
split_index = int(len(X)*.80)
train_x,val_x = X[:split_index],X[split_index:]
train_y,val_y = Y[:split_index],Y[split_index:]

In [0]:
class CNNDot(Model):

    def __init__(self, _parameter_dict, _word_to_id, _device, _pointwise=False, _debug=False):

        self.debug = _debug
        self.parameter_dict = _parameter_dict
        self.device = _device
        self.pointwise = _pointwise
        self.word_to_id = _word_to_id
        self.number_of_labels = self.parameter_dict['number_of_labels']

        if self.debug:
            print("Init Models")

        self.encoder = com.CNN(_vectors=self.parameter_dict['vectors'], _vocab_size=self.parameter_dict['vocab_size'] ,
                           _embedding_dim = self.parameter_dict['embedding_dim'] , _output_dim = self.parameter_dict['output_dim'],_debug=self.debug).to(self.device)
        
        self.dense = com.DenseClf(inputdim= self.parameter_dict['output_dim'],        # *2 because we have two things concatinated here
                                  hiddendim= self.parameter_dict['output_dim']/2,
                                  outputdim=self.number_of_labels).to(self.device)
        
        
    def train(self, data, optimizer, loss_fn, device):
    #
        if self.pointwise:
            return self._train_pointwise_(data, optimizer, loss_fn, device)
        else:
            return self._train_pairwise_(data, optimizer, loss_fn, device)

    def _train_pairwise_(self, data, optimizer, loss_fn, device):
        '''
            Given data, passes it through model, inited in constructor, returns loss and updates the weight
            :params data: {batch of question, pos paths, neg paths and dummy y labels}
            :params optimizer: torch.optim object
            :params loss fn: torch.nn loss object
            :params device: torch.device object

            returns loss
        '''

        # Unpacking the data and model from args
        sent_batch, y_label = data['sent_batch'], data['y_label']

        optimizer.zero_grad()

        # Encoding all the data
#         hidden = self.encoder.init_hidden(sent_batch.shape[0],self.device)
        sent_batch_encoded = self.encoder(sent_batch)
        if True:
            print(sent_batch_encoded.shape)
        # Calculating dot score
        pred = self.dense(sent_batch_encoded)


        '''
            If `y == 1` then it assumed the first input should be ranked higher
            (have a larger value) than the second input, and vice-versa for `y == -1`
        '''
        try:
#             print(torch.max(pred), torch.max(y_label))
            loss = loss_fn(pred, y_label.float())
        except RuntimeError:
            traceback.print_exc()
            print(torch.max(pred.to(cpu)), torch.max(y_label.to(cpu)))
#             print(pos_scores.shape, neg_scores.shape, y_label.shape,  ques_batch.shape, pos_batch.shape, neg_batch.shape)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), .5)
        optimizer.step()
        return loss



    def predict(self, sent, device):
        """
            Same code works for both pairwise or pointwise
        """
        with torch.no_grad():

            self.encoder.eval()
            self.dense.eval()
            
#             hidden = self.encoder.init_hidden(sent.shape[0], self.device)
            sent_encoded = self.encoder(sent.long())
            pred = self.dense(sent_encoded)


            self.encoder.train()
            self.dense.train()
            return pred

In [0]:
!wget 'https://s3.eu-central-1.amazonaws.com/maastrichtuniversity-ids-open/metadata/glove.42B.300d.txt'

In [25]:
vecs = np.random.randn(len(word_index),300)
embeddings_index = {}
f = open('glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))




Found 0 word vectors.


In [0]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
vecs = embedding_matrix

In [0]:
parameter_dict = {
    'number_of_layer' : 1,
    'bidirectional' : True,
    'embedding_dim' : 300,
    'max_length' : MAX_SEQUENCE_LENGTH,
    'hidden_size':256,
    'vocab_size':len(word_index),
    'dropout':0.2,
    'vectors':vecs,
    'number_of_labels' : len(all_class),
    'output_dim' : split_index
}

In [29]:
np.mean([len(text) for text in texts])

311.41740226986127

In [0]:
loss_fn = nn.BCELoss()
modeler = CNNDot( _parameter_dict = parameter_dict,_word_to_id=None,
                                     _device=device,_pointwise=False, _debug=False)

optimizer = optim.Adam(list(filter(lambda p: p.requires_grad, modeler.encoder.parameters()))+
                       list(filter(lambda p: p.requires_grad, modeler.dense.parameters())))

In [0]:
class SimplestSampler:
    """
        Given X and Y matrices (or lists of lists),
            it returns a batch worth of stuff upon __next__
    :return:
    """

    def __init__(self, data, bs: int = 64):

        try:
            assert len(data["x"]) == len(data["y"])
        except AssertionError:

            raise MismatchedDataError(f"Length of x is {len(data['x'])} while of y is {len(data['y'])}")

        self.x = data["x"]
        self.y = data["y"]
        self.n = len(self.x)
        self.bs = bs  # Batch Size

    def __len__(self):
        return self.n // self.bs - (1 if self.n % self.bs else 0)

    def __iter__(self):
        self.i, self.iter = 0, 0
        return self

    def __next__(self):
        if self.i + self.bs >= self.n:
            raise StopIteration

        _x, _y = self.x[self.i:self.i + self.bs], self.y[self.i:self.i + self.bs]
        self.i += self.bs

        return _x, _y

In [0]:
train_data = {
    'x' : train_x,
    'y': np.asarray(train_y)
}

valid_data = {
    'x' : val_x,
    'y' : np.asarray(val_y)
}

train_fact_iter = SimplestSampler(train_data)
valid_fact_iter = SimplestSampler(valid_data)

In [33]:
for dat in train_fact_iter:
    break
dat[0].shape, dat[1].shape

((64, 200), (64, 578))

In [34]:
_x = torch.tensor(dat[0], dtype=torch.long, device=device)
_y = torch.tensor(dat[1], dtype=torch.long, device=device)

data = {
    'sent_batch' : _x,
    'y_label' : _y
}

loss = modeler.train(data, optimizer, loss_fn, device)
op = modeler.predict(_x,device)

torch.Size([64, 634])


In [35]:
loss

tensor(0.6943, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

In [0]:
def eval(op,_y):
    return torch.mean(op.round()[_y==1])

In [0]:
def simplest_loop(epochs: int,
                  train_data_iter,
                  valid_data_iter,
                  device: torch.device,
                  model,
                  optimizer,
                  loss_fn,
                  eval_fn= None) -> (list, list, list):
    """
        A fn which can be used to train a language model.
        The model doesn't need to be an nn.Module,
            but have an eval (optional), a train and a predict function.
        Data should be a dict like so:
            {"train":{"x":np.arr, "y":np.arr}, "val":{"x":np.arr, "y":np.arr} }
        Train_fn must return both loss and y_pred
        :param epochs: number of epochs to train for
        :param data: a dict having keys train_x, test_x, train_y, test_y
        :param device: torch device to create new tensor from data
        :param opt: optimizer
        :param loss_fn: loss function
        :param train_fn: function to call with x and y
        :param predict_fn: function to call with x (test)
        :param data_fn: a class to which we can pass X and Y, and get an iterator.
        :param eval_fn: (optional) function which when given pred and true, returns acc
        :return: traces
    """

    train_loss = []
    train_acc = []
    val_acc = []
    lrs = []

    # Epoch level
    for e in range(epochs):

        per_epoch_loss = []
        per_epoch_tr_acc = []

        # Train
        with Timer() as timer:

            # Make data

            for x, y in tqdm(train_data_iter):
                optimizer.zero_grad()

                _x = torch.tensor(x, dtype=torch.long, device=device)
                _y = torch.tensor(y, dtype=torch.long, device=device)
                
                data = {
                    'sent_batch' : _x,
                    'y_label' : _y
                }
                
                loss = model.train(data, optimizer, loss_fn, device)
                y_pred = model.predict(_x,device)

                per_epoch_tr_acc.append(eval_fn(y_pred,_y).item())
                per_epoch_loss.append(loss.item())


        # Val
        with torch.no_grad():

            per_epoch_vl_acc = []
            for x, y in tqdm(valid_data_iter):
                _x = torch.tensor(x, dtype=torch.long, device=device)
                _y = torch.tensor(y, dtype=torch.long, device=device)

                y_pred = model.predict(_x,device)

                per_epoch_vl_acc.append(eval_fn(y_pred, _y).item())

        # Bookkeep
        train_acc.append(np.mean(per_epoch_tr_acc))
        train_loss.append(np.mean(per_epoch_loss))
        val_acc.append(np.mean(per_epoch_vl_acc))

        print("Epoch: %(epo)03d | Loss: %(loss).5f | Tr_c: %(tracc)0.5f | Vl_c: %(vlacc)0.5f | Time: %(time).3f min"
              % {'epo': e,
                 'loss': float(np.mean(per_epoch_loss)),
                 'tracc': float(np.mean(per_epoch_tr_acc)),
                 'vlacc': float(np.mean(per_epoch_vl_acc)),
                 'time': timer.interval / 60.0})

    return train_acc, train_loss, val_acc

In [38]:
traces = simplest_loop(epochs= 150,
                  train_data_iter=train_fact_iter,
                  valid_data_iter=valid_fact_iter,
                  device=device,
                  model = modeler,
                  optimizer=optimizer,
                  loss_fn=loss_fn,
             eval_fn=eval)

 25%|██▌       | 2/8 [00:00<00:00, 11.80it/s]

torch.Size([64, 634])
torch.Size([64, 634])
torch.Size([64, 634])


 50%|█████     | 4/8 [00:00<00:00, 11.96it/s]

torch.Size([64, 634])
torch.Size([64, 634])
torch.Size([64, 634])


100%|██████████| 8/8 [00:00<00:00, 11.82it/s]

...

torch.Size([64, 634])
torch.Size([64, 634])
torch.Size([64, 634])


 88%|████████▊ | 7/8 [00:00<00:00, 11.23it/s]

torch.Size([64, 634])
torch.Size([64, 634])


9it [00:00,  9.40it/s]                       
2it [00:00, 72.67it/s]               

torch.Size([64, 634])
Epoch: 149 | Loss: 0.00550 | Tr_c: 0.80906 | Vl_c: 0.10378 | Time: 0.014 min


In [0]:
def get_predictions_given_text(abstract):
    global modeler, all_class_to_id, word_index
    
    text_id = np.asarray([word_index.get(word, 0) for word in abstract.split()])
    _x = torch.tensor(text_id, dtype=torch.long, device=device).unsqueeze(0)
    y_pred = modeler.predict(_x, device)
    classes = [all_class[i] for i,x in enumerate(y_pred.round().int().squeeze()) if x.item() == 1]
    terms = get_list_for_terms(classes)
    return terms

In [0]:
y_actual = []
for result in val_y:
  idx = np.where(result==1)
  y_actual.append(np.take((all_class), idx))

In [0]:
type(y_actual[0][0])

numpy.ndarray

In [0]:
actual_labels = []
for i in y_actual:
  for j in i:
    #y = j.tolist()
    #y+= y.split('+')
    #print("i: ", i," j: ", j, " y: ", y)
    actual_labels.append(get_list_for_terms(j))

In [0]:
len(actual_labels)


159

In [0]:
val_data =  tokenizer.sequences_to_texts(val_x)

In [0]:
predictions = [get_predictions_given_text(data_points) for index, data_points in enumerate(val_data)]

In [0]:
predictions

[[],
 ['Breast Neoplasms',
  'Ovarian Neoplasms',
  'Hereditary Breast and Ovarian Cancer Syndrome'],
 [],
 ['Retinoblastoma'],
 ['Prader-Willi Syndrome'],
 ['Wiskott-Aldrich Syndrome'],
 [],
 ['Huntington Disease'],
 ['Lesch-Nyhan Syndrome'],
 ['Genetic Diseases, Inborn',
  'Muscular Dystrophies',
  'Myotonia',
  'Muscular Diseases',
  'Myotonic Dystrophy'],
 ['Breast Neoplasms',
  'Ovarian Neoplasms',
  'Hereditary Breast and Ovarian Cancer Syndrome',
  'Breast Neoplasms, Male'],
 [],
 ['Genetic Diseases, Inborn'],
 ['Myotonic Dystrophy'],
 [],
 ['Breast Neoplasms',
  'Ovarian Neoplasms',
  'Hereditary Breast and Ovarian Cancer Syndrome'],
 ['Huntington Disease'],
 ['Ataxia Telangiectasia'],
 ['Neoplasms'],
 [],
 [],
 ['Muscular Diseases', 'Myotonic Dystrophy', 'Muscular Dystrophies'],
 [],
 [],
 ['Breast Neoplasms', 'Neoplasms'],
 ['Muscular Dystrophy, Emery-Dreifuss'],
 [],
 ['Breast Neoplasms',
  'Hereditary Breast and Ovarian Cancer Syndrome',
  'Neoplasms'],
 ['Colorectal Neopla

In [0]:
val_data

In [0]:
string1 = val_data
string2 = predictions 
string3 = actual_labels

with open ("results_cnn_disease_names.csv","w") as f:
  writer = csv.writer(f)
  writer.writerows(zip(string1,string2,string3))